Notebook pour la validation de la métrique proposée dans le papier d'Amazon.

# 1 - Import des librairies

In [5]:
import hashlib
import requests
from pathlib import Path
import tomllib
import pandas as pd
from io import StringIO
import re
from typing import Optional, Dict, Any
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
import slugify
import ollama

# 2 - Chargement des données et traitement préliminaire

On recupère depuis data.gouv.fr le jeu de données des avis sur la fiscalité en France.

In [ ]:
# Define the URL and expected SHA-1 checksum
url = "https://www.data.gouv.fr/fr/datasets/r/bc085888-e6bd-445d-b3f4-632190c29e3f"
expected_sha1 = "90540350af64eb61f8a9823c83468934b19634c1"

# Define the target directory and file path
data_dir = Path("../data")
data_dir.mkdir(parents=True, exist_ok=True)
file_path = data_dir / "01_251021_AvisFiscalite.csv"

# Check if the file already exists
if file_path.exists():
    print(f"File already exists: {file_path}")
else:
    # Download the file if it doesn't exist
    print(f"Downloading file: {file_path}")
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad HTTP responses
    file_path.write_bytes(response.content)

    # Verify the SHA-1 checksum
    sha1 = hashlib.sha1()
    with file_path.open("rb") as f:
        while chunk := f.read(8192):
            sha1.update(chunk)
    calculated_sha1 = sha1.hexdigest()
    if calculated_sha1 == expected_sha1:
        print("SHA-1 checksum verified successfully.")
    else:
        print(f"SHA-1 checksum mismatch! Expected: {expected_sha1}, Got: {calculated_sha1}")

File already exists: ..\data\raw\fiscalite_et_les_depenses_publiques.csv


On transforme les données en un dataframe pandas et on ne garde que les colonnes utiles.
Nous nous intéressons à la question 'QUXVlc3Rpb246MTYz - Que faudrait-il faire pour rendre la fiscalité plus juste et plus efficace ?'

In [7]:
df = pd.read_csv(file_path, sep=",")
col_name = "QUXVlc3Rpb246MTYz - Que faudrait-il faire pour rendre la fiscalité plus juste et plus efficace ?"
df_contrib = df[['authorId', col_name]].rename(
    {col_name:'contribution'},
    axis=1)
df_contrib.head(10)

C:\Users\garan\AppData\Local\Temp\ipykernel_29820\262606293.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=",")


,authorId,contribution
0,VXNlcjo3ZTVjYTUwMi0xZDZlLTExZTktOTRkMi1mYTE2M2...,NaN
1,VXNlcjo5NmNhYWM4ZS0xZTIwLTExZTktOTRkMi1mYTE2M2...,NaN
2,VXNlcjo3ZTVjYTUwMi0xZDZlLTExZTktOTRkMi1mYTE2M2...,NaN
3,VXNlcjpjNDY0ZjllMy0xZDk4LTExZTktOTRkMi1mYTE2M2...,Repartir les richesses. suppression de la tax...
4,VXNlcjo3MDdkM2IzOC0xZDYxLTExZTktOTRkMi1mYTE2M2...,"Les droits soient automatiques, comme nos devo..."
5,VXNlcjo3NmI3NTM2MS0xYjI2LTExZTktOTRkMi1mYTE2M2...,NaN
6,VXNlcjo2YTgwZTNmYi0xZTIxLTExZTktOTRkMi1mYTE2M2...,NaN
7,VXNlcjoxZTNlOTExYi0xZTIwLTExZTktOTRkMi1mYTE2M2...,réduire drastiquement la fraude fiscale. Impos...
8,VXNlcjo1ODljMWRiMy0xZDVhLTExZTktOTRkMi1mYTE2M2...,diminuer le taux de prelevement pour les retra...
9,VXNlcjo0OTUzNmNmYy0xZTIwLTExZTktOTRkMi1mYTE2M2...,TOUT FRANÇAIS DEVRA PAYER L’IMPÔT QU’IL SOIT D...


Vérifions combien de contributions nous avons en réalité.

In [8]:
df_contrib['contribution'].notna().sum()

np.int64(154140)

Il y a 154140 contributions non nulles.

In [9]:
df_contrib = df_contrib.dropna(subset=['contribution'])

# 3 - Extraction des idées via LLM

### Chargement du prompt

In [ ]:
with open("01_251021_Prompt.toml", "rb") as f:
    toml_data = tomllib.load(f)

SYSTEM_PROMPT = toml_data["prompt"]["system"]
USER_TEMPLATE = toml_data["prompt"]["user"]  # e.g. "<<< {input} >>>"

### Fonction pour utuliser le LLM avec Ollama

In [18]:
# Fonction pour construire les messages à envoyer au LLM
def build_messages(text: str):
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_TEMPLATE.format(input=text)},
    ]

class LLMBadCSV(Exception):
    pass

# Fonction pour nettoyer les balises de code et autres ajouts indésirables
def strip_code_fences(s: str) -> str:
    # Supprime les balises de code Markdown
    s = s.strip()
    s = re.sub(r"^```[a-zA-Z]*\s*", "", s)
    s = re.sub(r"\s*```$", "", s)
    # Supprime tout préfixe avant "CSV:" si ça arrive
    idx = s.find("CSV:")
    if idx != -1:
        s = s[idx:]
    return s

# Fonction pour extraire le bloc CSV du texte retourné par le LLM
def extract_csv_block(s: str) -> str:
    s = strip_code_fences(s)
    # On vérifie si la sortie commence par "CSV:" comme on s'y attendd'après le prompt
    if s.startswith("CSV:"):
        return s[len("CSV:"):] # On retourne tout ce qui suit "CSV:"
    # Sinon, on tente de récupérer les lignes à partir de l'entête demandée dans le prompt
    m = re.search(r"(?mi)^description,type,syntax,semantic\s*$", s)
    if m:
        return s[m.start():] # On retourne tout à partir de l'entête
    return s  # Sinon on retourne tout le texte

# Fonction pour normaliser une ligne du CSV si les contraintes ne sont pas respectées
def normalize_row(row: Dict[str, Any]) -> Dict[str, Any]:
    # Normalise les valeurs attendues
    row["type"] = str(row.get("type","")).strip().lower()
    row["syntax"] = str(row.get("syntax","")).strip().lower()
    row["semantic"] = str(row.get("semantic","")).strip().lower()

    # Contraintes
    type_ok = {"statement", "proposition"}
    syntax_ok = {"positive", "negative"}
    semantic_ok = {"positive", "negative", "neutral"}

    # Normalisation si besoin
    if row["type"] not in type_ok:
        row["type"] = "statement" 
    if row["syntax"] not in syntax_ok:
        row["syntax"] = "positive"
    if row["semantic"] not in semantic_ok:
        row["semantic"] = "neutral"
    return row

# Fonction pour parser le CSV retourné par le LLM en DataFrame pandas
def parse_llm_csv(csv_text: str) -> pd.DataFrame:
    csv_text = csv_text.strip()
    if not csv_text.lower().startswith("description,type,syntax,semantic"):
        # Parfois le modèle met des espaces, on nettoie la première ligne
        lines = csv_text.splitlines()
        if lines:
            header = lines[0].replace(" ", "")
            if header.lower() == "description,type,syntax,semantic":
                lines[0] = "description,type,syntax,semantic"
                csv_text = "\n".join(lines)

    try:
        df = pd.read_csv(StringIO(csv_text), dtype=str, keep_default_na=False)
    except Exception as e:
        raise LLMBadCSV(f"CSV illisible: {e}")

    # Colonnes minimales
    expected_cols = ["description", "type", "syntax", "semantic"]
    missing = [c for c in expected_cols if c not in df.columns]
    if missing:
        raise LLMBadCSV(f"Colonnes manquantes: {missing}")

    # Normalisation
    df = df[expected_cols].copy()
    df = df.apply(lambda r: pd.Series(normalize_row(r.to_dict())), axis=1)
    return df

# Fonction principale pour appeler le LLM et obtenir un DataFrame
def call_llm_return_df(text: str) -> pd.DataFrame:
    messages = build_messages(text)
    resp = ollama.chat(
    model="llama3:8b-instruct-q4_K_M",
    messages=messages,
    options={
        "num_ctx": 2048, 
        "num_batch": 4,
        "temperature": 0,
        "top_p": 0.95,
        "seed": 42,
    }
    )

    raw = resp["message"]["content"]
    csv_block = extract_csv_block(raw)
    return parse_llm_csv(csv_block)

In [19]:
def extract_ideas_from_df(df_contrib: pd.DataFrame,
                          text_col: str = "contribution",
                          id_col: str = "authorId") -> pd.DataFrame:
    rows = []

    for i, row in tqdm(df_contrib.iterrows(), total=len(df_contrib), desc="LLM extraction"):
        text = str(row[text_col]).strip()
        auth = row[id_col]
        if not text:
            continue

        try:
            ideas_df = call_llm_return_df(text)
        except Exception as e:
            # On enregistre une ligne "échec" minimale pour traçabilité
            ideas_df = pd.DataFrame([{
                "description": f"[PARSE_FAIL] {str(e)[:200]}",
                "type": "statement",
                "syntax": "positive",
                "semantic": "neutral"
            }])

        # Ajoute le contexte
        ideas_df = ideas_df.copy()
        ideas_df.insert(0, "authorId", auth)
        ideas_df.insert(1, "contrib_index", i)
        rows.append(ideas_df)

    if not rows:
        return pd.DataFrame(columns=["authorId","contrib_index","description","type","syntax","semantic"])
    out = pd.concat(rows, ignore_index=True)
    return out

In [23]:
result = extract_ideas_from_df(df_contrib[0:200])  # test sur un petit échantillon

LLM extraction: 100%|██████████| 200/200 [42:23<00:00, 12.72s/it]  


In [26]:
result.head(10)

,authorId,contrib_index,description,type,syntax,semantic
0,VXNlcjpjNDY0ZjllMy0xZDk4LTExZTktOTRkMi1mYTE2M2...,3,Suppression de la taxe d'habitation pour tous ...,statement,negative,neutral
1,VXNlcjpjNDY0ZjllMy0xZDk4LTExZTktOTRkMi1mYTE2M2...,3,Réindexation des retraites via la CSG,proposition,positive,neutral
2,VXNlcjpjNDY0ZjllMy0xZDk4LTExZTktOTRkMi1mYTE2M2...,3,Élimination de la division entre les Français ...,proposition,positive,neutral
3,VXNlcjpjNDY0ZjllMy0xZDk4LTExZTktOTRkMi1mYTE2M2...,3,Les patrons ont besoin des plus modestes et vi...,statement,negative,neutral
4,VXNlcjpjNDY0ZjllMy0xZDk4LTExZTktOTRkMi1mYTE2M2...,3,Les Français veulent vivre dignement de leur s...,proposition,positive,positive
5,VXNlcjo3MDdkM2IzOC0xZDYxLTExZTktOTRkMi1mYTE2M2...,4,Les droits soient automatiques,statement,negative,negative
6,VXNlcjo3MDdkM2IzOC0xZDYxLTExZTktOTRkMi1mYTE2M2...,4,comme nos devoirs de payer les impôts,statement,positive,neutral
7,VXNlcjoxZTNlOTExYi0xZTIwLTExZTktOTRkMi1mYTE2M2...,7,Réduire drastiquement la fraude fiscale,statement,negative,negative
8,VXNlcjoxZTNlOTExYi0xZTIwLTExZTktOTRkMi1mYTE2M2...,7,Imposer les grands groupes (GAFA) qui ne le so...,proposition,positive,negative
9,VXNlcjoxZTNlOTExYi0xZTIwLTExZTktOTRkMi1mYTE2M2...,7,Renforcer la taxe sur les transactions financi...,proposition,positive,neutral


In [27]:
# Define the target directory and file path
data_dir = Path("../data/processed")
data_dir.mkdir(parents=True, exist_ok=True)
result.to_csv("../data/processed/extraction_idees_principales.csv", index=False)

# 4 - Calcul de la métrique proposée

In [28]:
import numpy as np
from sentence_transformers import SentenceTransformer

In [ ]:
EMB_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # modèle pour calculer les emmbeddings
DEVICE = "cpu" 

In [55]:
def cosine_similarity(u: np.ndarray, v: np.ndarray) -> float:
    """cos(u,v) avec u et v"""
    return float(np.dot(u, v)/ (np.linalg.norm(u) * np.linalg.norm(v) + 1e-12))

def embed_texts(model: SentenceTransformer, texts: list[str], device: str = DEVICE) -> np.ndarray:
    """Embeddings (np.array) pour une liste de textes, normalisés."""
    embs = model.encode(texts, device=device, convert_to_numpy=True, normalize_embeddings=True, show_progress_bar=False)
    return embs


In [59]:
def compute_C_scores(
    df_contrib: pd.DataFrame,
    result: pd.DataFrame,
    text_col: str = "contribution",
    index_col: str = "contrib_index",
    sep: str = " || "
) -> pd.DataFrame:

    # Aggrégation des idées extraites dans un seul texte par contribution pour faire la comparaison
    ideas_grouped = (
        result
        .groupby(index_col, as_index=False)
        .agg(
            n_ideas=("description", "size"),
            ideas_text=("description", lambda s: sep.join([str(x).strip() for x in s if str(x).strip() != ""]))
        )
    )

    # Aligne avec df_contrib pour récupérer toutes les informations dans le df merged
    dfc = df_contrib.reset_index(drop=False).rename(columns={"index": index_col})
    merged = ideas_grouped.merge(
        dfc[[index_col, "authorId", text_col]],
        on=index_col, how="left"
    ).dropna(subset=[text_col]).copy()

    # Modèle d’embedding
    model = SentenceTransformer(EMB_MODEL)

    # Calcul des emmbeddings pour la contribution
    E_contrib = embed_texts(model, merged[text_col].tolist(), device=DEVICE)

    # Calcul des emmbeddings pour les idées
    E_ideas = embed_texts(model, merged["ideas_text"].fillna("").tolist(), device=DEVICE)
    
    # Calcul de la métrique pour chaque contribution
    C = np.zeros(E_ideas.shape[0])
    for i in range(E_ideas.shape[0]):
        Ci = cosine_similarity(E_contrib[i], E_ideas[i])
        C[i] = Ci
    
    # Construit la sortie avec les colonnes demandées
    out = pd.DataFrame({
        "authorId": merged["authorId"].values,
        "contrib_index": merged[index_col].values,
        "contribution": merged[text_col].values,
        "ideas_text": merged["ideas_text"].fillna("").values,
        "C": C,
        "n_ideas": merged["n_ideas"].values,
        "len_contrib": merged[text_col].fillna("").map(len).values,
        "len_ideas_text": merged["ideas_text"].fillna("").map(len).values,
    })

    return out.sort_values(["contrib_index"]).reset_index(drop=True)

In [60]:
scores = compute_C_scores(df_contrib, result)
scores.head()

,authorId,contrib_index,contribution,ideas_text,C,n_ideas,len_contrib,len_ideas_text
0,VXNlcjpjNDY0ZjllMy0xZDk4LTExZTktOTRkMi1mYTE2M2...,3,Repartir les richesses. suppression de la tax...,Suppression de la taxe d'habitation pour tous ...,0.920268,5,372,313
1,VXNlcjo3MDdkM2IzOC0xZDYxLTExZTktOTRkMi1mYTE2M2...,4,"Les droits soient automatiques, comme nos devo...",Les droits soient automatiques || comme nos de...,0.984356,2,69,71
2,VXNlcjoxZTNlOTExYi0xZTIwLTExZTktOTRkMi1mYTE2M2...,7,réduire drastiquement la fraude fiscale. Impos...,Réduire drastiquement la fraude fiscale || Imp...,0.986165,3,159,162
3,VXNlcjo1ODljMWRiMy0xZDVhLTExZTktOTRkMi1mYTE2M2...,8,diminuer le taux de prelevement pour les retra...,Diminuer le taux de prélèvement pour les retra...,0.489061,4,79,463
4,VXNlcjo0OTUzNmNmYy0xZTIwLTExZTktOTRkMi1mYTE2M2...,9,TOUT FRANÇAIS DEVRA PAYER L’IMPÔT QU’IL SOIT D...,Les Français devraient payer l'impôt qu'ils so...,0.954236,9,967,932


In [ ]:
scores.to_csv("../data/01_ExtractionsIdees_251021.csv", index=False)

In [62]:
scores["C"].describe()

count    200.000000
mean       0.782215
std        0.230390
min        0.022147
25%        0.739208
50%        0.854111
75%        0.930629
max        1.000000
Name: C, dtype: float64

# 5 - Mise en forme dans un CSV pour annotation manuelle